## Chatbot using langchain

## load data (example load UBER data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers
!pip install -q langchain-chroma
!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchainhub
!pip install -q gpt4all
!pip install unstructured
!pip install git+https://github.com/nltk/nltk.git@3.8.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import WebBaseLoader, UnstructuredHTMLLoader

In [ ]:
# NOTE: the code examples assume you're operating within a Jupyter notebook.
# download files
!mkdir data
!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1" -O data/UBER.zip
!unzip data/UBER.zip -d data

--2024-11-17 02:47:33--  https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/vetj6tgcux8e309swquxs/UBER.zip?rlkey=oy4vx60rplorounkh9wh2waux&dl=1 [following]
--2024-11-17 02:47:33--  https://www.dropbox.com/scl/fi/vetj6tgcux8e309swquxs/UBER.zip?rlkey=oy4vx60rplorounkh9wh2waux&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc364e638424d68113163c29f78d.dl.dropboxusercontent.com/cd/0/inline/CegEU00bnYZwfl-EqkuzzKvjimOUn3RWAM9JSigNw5I7Fdj_t7NY9x2WdIx_k0PetNwdMhIvrU-PA9PGg7sljdX12RWyIR7OrT1fsf0MG74AyIBCM5r7Bxj322Iycif8OAo/file?dl=1# [following]
--2024-11-17 02:47:34--  https://uc364e638424d68113163c29f78d.dl.dropboxusercontent.com/cd/0/inline/CegEU00bnYZ

In [ ]:
%pip install -qU  nest_asyncio

# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
file_paths = sorted(os.listdir("/content/data/UBER"))
all_docs = []
years = [2019, 2020, 2021, 2022]
for path,year in zip(file_paths,years):
    loader = UnstructuredHTMLLoader("/content/data/UBER/" + path)
    docs = loader.load()
    for doc in docs:
        doc.metadata["year"] = year
    all_docs.extend(docs)

In [ ]:
all_docs[0]

# Vectorize data using chroma

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-11-5d40e16e6074>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(
    documents=all_docs,
    embedding=embeddings
    ,persist_directory="/content/chroma_db",
)

## Retrieval
If you already have load data, you can start from this)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.vectorstores import Chroma

In [ ]:
# download ollama
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14313    0 14313    0     0  40364      0 --:--:-- --:--:-- --:--:-- 40318
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# run ollama in other subprocess
import subprocess
import time
import threading

# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

Ollama server is running in the background


In [ ]:
# pull model from ollama
# there are some option you can choose : "llama3.2", "llama3.1", "gemma2", "qwen2.5"
!ollama pull llama3.2:1b

[GIN] 2024/11/18 - 16:49:03 | 200 |      50.932µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 74701a8c35f6... 100% ▕▏ 1.3 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 4f659a1e86d7... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
llm = ChatOllama(model="llama3.2")
prompt = hub.pull("langchain-ai/chat-langchain-rephrase")
vectorstore = Chroma(persist_directory="/content/chroma_db/content/chroma_db", embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<ipython-input-12-f6d6d613e159>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="/content/chroma_db/content/chroma_db", embedding_function=embeddings)


In [ ]:
print(llm)

NameError: name 'llm' is not defined

# Test read pdf file

In [ ]:
!pip install langchain-ollama
!pip install pypdf

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
file_path = "/content/Guidance on Yolov5.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

[GIN] 2024/11/18 - 16:49:04 | 200 |  880.022432ms |       127.0.0.1 | POST     "/api/pull"


In [ ]:
# Prompt
from langchain_core.prompts import PromptTemplate

prompt_template = """Read the provided file and return the given answer
Only include information that is part of the document.
Do not include your own opinion or analysis.

Document:
"{document}"
{question}
Result:"""
prompt = PromptTemplate.from_template(prompt_template)


In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain.chains.llm import LLMChain
llm = ChatOllama(model="llama3.2:1b",max_tokens=512,
            top_p=0.95,
            temperature=0.4,)
llm_chain = LLMChain( prompt=prompt,llm=llm)

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="document"
)

In [ ]:
query = "which yolo version this document use"
result = stuff_chain.invoke({"question":query,"input_documents":docs})

[GIN] 2024/11/18 - 17:01:45 | 200 | 10.954695148s |       127.0.0.1 | POST     "/api/chat"


In [ ]:
result['output_text']

'The text appears to be a tutorial or guide for training and using YOLOv5 object detection model, specifically with labelImg tool. Here are the main points:\n\n**Training the Model**\n\n1. Prepare dataset by creating configuration file (.yaml) Helmet.yaml.\n2. Create weights file (best.pt).\n3. Run command python detect.py --img 640 --batch 4 --epochs 20 --data helmet.yaml --weights yolov5spt --cache.\n\n**Labeling Images**\n\n1. Download labelImg tool from GitHub.\n2. Initialize Anaconda environment with conda create -n labelimg_env.\n3. Activate the virtual environment using conda activate labelimg_env\n4. Navigate to source code directory of labelImg folder using cd < path_to_labelImg_folder >.\n5. Run python labelImg.py.\n\n**Labeling for Human Detection**\n\n1. Define predefined classes.txt file with human class in your dataset (e.g., person).\n2. Edit the file content with name of class in your dataset (e.g., "person").\n3. Download and install PyTorch and PyQt.\n4. Run labelImg 

## Test with UBER data

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
import re
def answer_query_with_year_filter(query):
    year_match = re.search(r"\b(20[1-2][0-9]\b)", query)
    if year_match:
        year = int(year_match.group(0))
    else:
        return "Please specify a year in your question from 2019-2022"

    meatadata_filter = {'year':year}

    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"filter": meatadata_filter},k=3)

    custom_prompt_template = """
    You are a knowledgeable assistant with access to a database of information. Use previous conversation history and the provided context to give relevant answers.

    Chat History:
    {history}

    Context:
    {context}

    Question: {question}
    Answer:"""
    prompt = PromptTemplate(
    input_variables=["history", "context", "question"],  # Include context for retrieved documents
    template=custom_prompt_template
    )
    memory = ConversationBufferMemory(memory_key="history", input_key="question")

    qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Adjust chain type if needed
    retriever=retriever,
    chain_type_kwargs={
        "prompt": prompt,
        "memory": memory,
        "document_variable_name": "context"  # Map retrieved documents to "context" in prompt
    }
    )

    response = qa_chain({"query": query})
    return response["result"]

In [ ]:
# test in single query
query = "What were some of the biggest risk factors for Uber in 2020?"
resopnse = answer_query_with_year_filter(query)
print(resopnse)

<ipython-input-14-c0bab01bbc49>:32: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", input_key="question")
<ipython-input-14-c0bab01bbc49>:45: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})


INFO [main] starting c++ runner | tid="135635242602496" timestamp=1730125737
INFO [main] build info | build=10 commit="3a8c75e" tid="135635242602496" timestamp=1730125737
INFO [main] system info | n_threads=1 n_threads_batch=1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | RISCV_VECT = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="135635242602496" timestamp=1730125737 total_threads=2
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="39065" tid="135635242602496" timestamp=1730125737
INFO [main] model loaded | tid="135635242602496" timestamp=1730125745
INFO [update_slots] input truncated | n_ctx=2048 n_erase=93073 n_keep=4 n_left=2044 n_shift=1022 tid="135635242602496" timestamp=1730125746
[GIN] 2024/10/28 - 14:29:12 | 200 | 15.381672472s |       127.0.0.1 | POS

In [ ]:
# test in
while True:
    user_query = input("User: ")
    if user_query.lower() in ["exit", "quit"]:
        break
    response = answer_query_with_year_filter(user_query)
    print(f"Assistant: {response}")

## API using Chainlit for UBER

In [ ]:
!pip install chainlit pyngrok
!pip install -U langchain-ollama langchain-chroma langchain-huggingface langchain_community

In [ ]:
import re
import chainlit as cl

In [ ]:
# config.py
%%writefile config.py
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama.chat_models import ChatOllama
# Initialize LLM
llm = ChatOllama(model="llama3.2")

# Initialize embeddings for Chroma vectorstore
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Initialize Chroma vectorstore with the embedding function
vectorstore = Chroma(persist_directory="/content/chroma_db/content/chroma_db", embedding_function=embedding)

Overwriting config.py


In [ ]:
%%writefile app.py
import chainlit as cl
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM  # Updated import for Ollama
from langchain.memory import ConversationBufferMemory
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from config import *

@cl.on_startup
def start():
    cl.Message(content="I an assistant was built to answer UBER report data from 2019-2022. How can I help you")

@cl.on_message
async def main(message: cl.Message):
    # Ensure message content is a string
    if not isinstance(message.content, str):
        await cl.Message(content="Invalid input. Please provide a text message.").send()
        return

    # Set up year filtering
    year_match = re.search(r"\b(20[1-2][0-9]\b)", message.content)
    if year_match:
        year = int(year_match.group(0))
    else:
        await cl.Message(content="Please specify a year in your question from 2019-2022").send()
        return

    metadata_filter = {'year': year}

    # Set up the vectorstore retriever with the metadata filter
    retriever = vectorstore.as_retriever(
        search_type="similarity", search_kwargs={"filter": metadata_filter}, k=3
    )

    # Custom prompt and memory
    custom_prompt_template = """
    You are a knowledgeable assistant with access to a database of information. Use previous conversation history and the provided context to give relevant answers.

    Chat History:
    {history}

    Context:
    {context}

    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=custom_prompt_template
    )

    memory = ConversationBufferMemory(memory_key="history", input_key="question")

    # Set up the RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": prompt,
            "memory": memory,
            "document_variable_name": "context"
        }
    )

    # Generate response
    response = qa_chain({"query": message.content})
    await cl.Message(content=response["result"]).send()


Overwriting app.py


In [ ]:
!chainlit run app.py &>/content/logs.txt &

In [ ]:
!ngrok config add-authtoken "2lsgvMrlHIJ6qHbrQScddkb5hWD_2c5oZqxPf2mLJr2i1dcT8"

In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# using bookdata

# test using speech recognition

In [ ]:
!pip install speechrecognition

In [ ]:
import speech_recognition as sr
def recognize_speech():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source :
        print("Talk")
        audio_text = recognizer.listen(source)
        print("Time over, thanks")
        # recoginze_() method will throw a request
        # error if the API is unreachable,
        # hence using exception handling

        try:
            # using google speech recognition
            print("Text: "+recognizer.recognize_google(audio_text))
            return recognizer.recognize_google(audio_text)
        except:
            print("Sorry, I did not get that")

In [ ]:
!pip install -r /content/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.5
    Uninstalling langchain-community-0.3.5:
      Successfully uninstalled langchain-community-0.3.5


In [ ]:
!unzip /content/chroma_db_pypdf.zip

Archive:  /content/chroma_db_pypdf.zip
   creating: content/chroma_db_pypdf/
  inflating: content/chroma_db_pypdf/chroma.sqlite3  
   creating: content/chroma_db_pypdf/b9fb5b8a-d057-4932-ac1e-23903f0a6b8f/
  inflating: content/chroma_db_pypdf/b9fb5b8a-d057-4932-ac1e-23903f0a6b8f/length.bin  
  inflating: content/chroma_db_pypdf/b9fb5b8a-d057-4932-ac1e-23903f0a6b8f/data_level0.bin  
  inflating: content/chroma_db_pypdf/b9fb5b8a-d057-4932-ac1e-23903f0a6b8f/link_lists.bin  
  inflating: content/chroma_db_pypdf/b9fb5b8a-d057-4932-ac1e-23903f0a6b8f/index_metadata.pickle  
  inflating: content/chroma_db_pypdf/b9fb5b8a-d057-4932-ac1e-23903f0a6b8f/header.bin  


In [ ]:
from pyngrok import ngrok

In [ ]:
!pip install --upgrade chainlit

In [ ]:
%%writefile app.py
import torch
from datetime import datetime, timedelta
from langchain_ollama.chat_models import ChatOllama
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
import chainlit as cl
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun
from typing import List
from langchain.schema.retriever import BaseRetriever
from langchain.schema.document import Document
import os
import speech_recognition as sr
def recognize_speech():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source :
        print("Talk")
        audio_text = recognizer.listen(source)
        print("Time over, thanks")
        # recoginze_() method will throw a request
        # error if the API is unreachable,
        # hence using exception handling

        try:
            # using google speech recognition
            print("Text: "+recognizer.recognize_google(audio_text))
            return recognizer.recognize_google(audio_text)
        except:
            print("Sorry, I did not get that")

class RerankRetriever(VectorStoreRetriever):
    vectorstore: Chroma

    def get_relevant_documents(self, query: str):

        initial_retriever = self.vectorstore.as_retriever( search_type="similarity",search_kwargs={"k": 10})
        docs = initial_retriever.get_relevant_documents(query=query)

        if not docs:
            return ''

        return docs
        # candidates = [doc.page_content for doc in docs]
        # queries = [query] * len(candidates)

        # input_ids = tokenizer_rerank(queries, candidates, padding=True, truncation=True, return_tensors="pt").to(device)
        # with torch.no_grad():
        #     scores = model_rerank(**input_ids, return_dict=True).logits.view(-1).float()
        #     sorted_scores, indices = torch.sort(scores, descending=True)

        # reranked_docs = [docs[i] for i in indices]

        # return reranked_docs[:min(3, len(reranked_docs))]


class LLMServe:
    def __init__(self):
        self.embeddings = self.load_embeddings()
        self.retriever = self.load_retriever(self.embeddings)
        self.llm = ChatOllama(model="llama3.2:1b")
        self.contextualize_q_prompt = self.setup_contextualize_q_prompt()
        self.qa_prompt = self.setup_qa_prompt()
        self.history_aware_retriever = create_history_aware_retriever(
            self.llm, self.retriever, self.contextualize_q_prompt
        )
        self.question_answer_chain  = create_stuff_documents_chain(self.llm, self.qa_prompt)
        self.rag_chain = create_retrieval_chain(self.history_aware_retriever, self.question_answer_chain )
        self.store = {}

        self.chat_history_path = "chat_history_logs"
        os.makedirs(self.chat_history_path, exist_ok=True)

    def save_history_to_html(self, session_id: str):
            """Save the current session chat history to an HTML file."""
            history = self.get_session_history(session_id).messages
            if not history:
                return

            file_path = os.path.join(self.chat_history_path, f"chat_history_{session_id}.html")

            # check file exist or not to initialize this content
            html_content =""
            if not os.path.exists(file_path):
                    html_content = "<html><head><title>Chat History</title></head><body>"
                    html_content += "<h2>Chat History</h2>"

            latest_chat = history[-2:]

            # for user
            html_content += f"<p><strong>HumanMessage:</strong> {latest_chat[0].content}</p>"
            # for AI
            html_content += f"<p><strong>AIMessage:</strong> {latest_chat[1].content}</p>"

            html_content += "</body></html>"

            with open(file_path, "a",encoding="utf-8") as f:
                f.write(html_content)

    def load_embeddings(self):
        model_name = "sentence-transformers/all-mpnet-base-v2"
        model_kwargs = {'device': 'cpu'}
        encode_kwargs = {'normalize_embeddings': False}
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        return embeddings

    def load_retriever(self, embeddings):
        db = Chroma(persist_directory="chroma_db_pypdf/content/chroma_db_pypdf", embedding_function=embeddings)
        retriever = RerankRetriever(vectorstore=db)

        return retriever

    def setup_contextualize_q_prompt(self):
        contextualize_q_system_prompt = (
            """Given a chat history and the latest user question \
            which might reference context in the chat history, formulate a standalone question \
            which can be understood without the chat history. Do NOT answer the question, \
            just reformulate it if needed and otherwise return it as is."""
        )
        return ChatPromptTemplate.from_messages(
            [
                ("system", contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )

    def setup_qa_prompt(self):
        system_prompt = (
            """You are an assistant for question-answering tasks. \
            Use the following pieces of retrieved context to answer the question. \
            If you don't know the answer, just say that you don't know. \
            Use three sentences maximum and keep the answer concise.\

            {context}"""
        )
        return ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        current_time = datetime.now()
        if session_id not in self.store:
            self.store[session_id] = {
                'history': ChatMessageHistory(),
                'last_accessed': current_time
            }

        if current_time - self.store[session_id]['last_accessed'] > timedelta(minutes=10):
            del self.store[session_id]
            self.store[session_id] = {
                'history': ChatMessageHistory(),
                'last_accessed': current_time
            }

        self.store[session_id]['last_accessed'] = current_time
        return self.store[session_id]['history']


    def conversational_rag_chain(self, question, session_id=(datetime.now()).strftime("%y%m%d%H%M")):
        conversational_rag_chain = RunnableWithMessageHistory(
            self.rag_chain,
            self.get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )
        answer = conversational_rag_chain.invoke(
            {"input": question},
            config={
                "configurable": {"session_id": session_id}
            },
        )["answer"]

        self.save_history_to_html(session_id)

        return answer

llm_serve = LLMServe()
toogle_mode = "text"

# def toogle_input_mode():
#     global toogle_mode
#     toogle_mode = "speech" if toogle_mode == "speech" else "text"
#     cl.log(f"Input mode switch to {toogle_mode}")

# cl.register_action(name="toogle_mode", description="Toggle between text and speech input", handler=toogle_input_mode)
def toogle_input_mode():
    """Toggle between speech and text mode."""
    global toogle_mode
    toogle_mode = "speech" if toogle_mode == "text" else "text"
    cl.log(f"Input mode switched to {toogle_mode}")


# # Register action explicitly
# cl.register_action(name="toogle_mode", description="Toggle between text and speech input", handler=toogle_input_mode)

@cl.on_message
async def handle_message(message: cl.Message):
    global toogle_mode

    #handle toogle mode
    if message.content.strip().lower() == "/toggle":
      toogle_input_mode()
      await cl.Message(content=f"Input mode is now {toogle_mode}.").send()

    if toogle_mode == "speech":
        user_input_speech = recognize_speech()

        if not user_input_speech:
            await cl.Message(content="I couldn't hear anything. Please try again.").send()
            return

        try:
            # Process the message with the LLMServe's conversational chain.
            response = llm_serve.conversational_rag_chain(question=user_input_speech)

            await cl.Message(content=response).send()

        except Exception as e:
            print(f"Error during processing: {str(e)}")
            await cl.Message(content=f"Error: {str(e)}").send()

    elif toogle_mode == "text":
        user_input = message.content.lower()

        if not isinstance(message.content, str):
            await cl.Message(content="Invalid input. Please provide a text message.").send()
            return

        try:
            # Process the message with the LLMServe's conversational chain.
            response = llm_serve.conversational_rag_chain(question=user_input)

            await cl.Message(content=response).send()

        except Exception as e:
            print(f"Error during processing: {str(e)}")
            await cl.Message(content=f"Error: {str(e)}").send()


# if __name__ == "__main__":
#     from chainlit.cli import run_chainlit
#     run_chainlit(__file__)

Overwriting app.py


In [ ]:
!chainlit run app.py &>/content/logs.txt &

In [ ]:
!ngrok config add-authtoken "2m4A63mKEZfYnDFMdwVGm02ET56_3i7caNEU2NLHzDurMKHwG"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# test app.py has error or not
!python app.py

/content/app.py:42: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class RerankRetriever(VectorStoreRetriever):
/content/app.py:114: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2024-11-17 03:44:02.860382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 03:44:02.980531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory 

In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://962e-35-198-204-97.ngrok-free.app
